In [1]:
from jesse import helpers, research

from custom_indicators.all_features import feature_matrix
from research_tools import TrailingStopLabel

warmup_1m, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2021-01-01"),
    helpers.date_to_timestamp("2024-12-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

In [2]:
trailing_stop_label = TrailingStopLabel(trading_1m, n_bar=15, k=1)
features = feature_matrix(trading_1m, sequential=True)

In [3]:
from copy import deepcopy

import numpy as np
import pandas as pd

SKIP_N = 240
TRAIN_TEST_SPLIT_INDEX = int((len(trailing_stop_label.labels) - SKIP_N) * 0.7)

x_train = pd.DataFrame(
    features.features[SKIP_N:TRAIN_TEST_SPLIT_INDEX], columns=features.names
)
y_train = trailing_stop_label.labels[SKIP_N:TRAIN_TEST_SPLIT_INDEX]

mask = deepcopy(y_train)
mask = mask != 0
y_train = y_train[mask]
y_train = pd.Series((y_train == 1).astype(np.int8), name="label")
x_train = x_train[mask]
x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

x_test = pd.DataFrame(
    features.features[TRAIN_TEST_SPLIT_INDEX + SKIP_N :], columns=features.names
)
y_test = trailing_stop_label.labels[TRAIN_TEST_SPLIT_INDEX + SKIP_N :]

mask = deepcopy(y_test)
mask = mask != 0
y_test = y_test[mask]
y_test = pd.Series((y_test == 1).astype(np.int8), name="label")
x_test = x_test[mask]
x_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1334848, 336)
(1334848,)
(604082, 336)
(604082,)


In [4]:
from featurewiz import FeatureWiz

fwiz = FeatureWiz(corr_limit=0.99, verbose=0)
X_train_selected, y_train = fwiz.fit_transform(x_train, y_train)
X_test_selected = fwiz.transform(x_test)
fwiz.features

Imported lazytransform v1.18. 

Imported featurewiz 0.5.83. Use the following syntax:
    >>> wiz = FeatureWiz(feature_engg = '', nrows=None, transform_target=True,
        		category_encoders="auto", auto_encoders='VAE', ae_options={},
        		add_missing=False, imbalanced=False, verbose=0)
    >>> X_train_selected, y_train = wiz.fit_transform(X_train, y_train)
    >>> X_test_selected = wiz.transform(X_test)
    >>> selected_features = wiz.features
    
featurewiz is given 1.0 as correlation limit...
    Skipping feature engineering since no feature_engg input...
    final list of category encoders given: ['label', 'label']
    final list of scalers given: []
Loaded input data. Shape = (1334848, 336)
#### Starting featurewiz transform for train data ####
    Single_Label Binary_Classification problem 
Shape of dataset: (1334848, 336). Now we classify variables into different types...
Time taken to define data pipeline = 5 second(s)
No model input given...
Lazy Transformer Pipeline c

['evenbetter_sinewave_long',
 'acc_swing_index_dt',
 'evenbetter_sinewave_short',
 'acc_swing_index_lag1',
 'evenbetter_sinewave_long_lag3',
 'bandpass_dt',
 'roofing_filter_ddt',
 'acc_swing_index',
 'ehlers_early_onset_trend_ddt',
 'highpass_bp_dt',
 'acc_swing_index_ddt',
 'acc_swing_index_lag2',
 'comb_spectrum_pwr_26',
 'roofing_filter_lag2',
 'hurst_coef_slow_lag2',
 'roofing_filter',
 'comb_spectrum_pwr_35',
 'phase_accumulation_dt',
 'comb_spectrum_pwr_11',
 'phase_accumulation_lag3',
 'roofing_filter_dt',
 'evenbetter_sinewave_long_lag1',
 'acp_pwr_0',
 'hurst_coef_fast',
 'comb_spectrum_pwr_33',
 'ac_42',
 'adaptive_cci_lag3',
 'ac_44',
 'hurst_coef_fast_lag3',
 'adaptive_bp_lead',
 'acp_pwr_5',
 'comb_spectrum_pwr_37',
 'phase_accumulation_ddt',
 'ac_27',
 'ac_1',
 'ac_35',
 'hurst_coef_slow_dt',
 'phase_accumulation_lag1',
 'swamicharts_stochastic_47',
 'comb_spectrum_pwr_12',
 'ac_28',
 'comb_spectrum_pwr_14',
 'swamicharts_stochastic_18',
 'comb_spectrum_pwr_28',
 'phase_

In [7]:
fwiz.features

['evenbetter_sinewave_long',
 'acc_swing_index_dt',
 'evenbetter_sinewave_short',
 'acc_swing_index_lag1',
 'evenbetter_sinewave_long_lag3',
 'bandpass_dt',
 'roofing_filter_ddt',
 'acc_swing_index',
 'ehlers_early_onset_trend_ddt',
 'highpass_bp_dt',
 'acc_swing_index_ddt',
 'acc_swing_index_lag2',
 'comb_spectrum_pwr_26',
 'roofing_filter_lag2',
 'hurst_coef_slow_lag2',
 'roofing_filter',
 'comb_spectrum_pwr_35',
 'phase_accumulation_dt',
 'comb_spectrum_pwr_11',
 'phase_accumulation_lag3',
 'roofing_filter_dt',
 'evenbetter_sinewave_long_lag1',
 'acp_pwr_0',
 'hurst_coef_fast',
 'comb_spectrum_pwr_33',
 'ac_42',
 'adaptive_cci_lag3',
 'ac_44',
 'hurst_coef_fast_lag3',
 'adaptive_bp_lead',
 'acp_pwr_5',
 'comb_spectrum_pwr_37',
 'phase_accumulation_ddt',
 'ac_27',
 'ac_1',
 'ac_35',
 'hurst_coef_slow_dt',
 'phase_accumulation_lag1',
 'swamicharts_stochastic_47',
 'comb_spectrum_pwr_12',
 'ac_28',
 'comb_spectrum_pwr_14',
 'swamicharts_stochastic_18',
 'comb_spectrum_pwr_28',
 'phase_

In [6]:
import json

with open("offline/fwiz.json", "w") as f:
    json.dump(sorted(fwiz.features), f)